# The Wealth of Cities
## Predicting the Wealth of a City from Satellite Imagery

## Introduction

Accurate measurements of the economic characteristics of cities critically influence research and government policy (Jean, Luo). Such measurements can shape decisions by governments on how to allocate resources and provide infrastructure to improve human livelihoods in a wide-range of situations. Although economic data is readily available for most developed and some developing nations, many regions of the modern world remain unexposed to the benefits of economic analysis because of a lack key measures of economic development and efficiency. Regions such as parts of Africa do not have systems in place to conduct economic surveys or other means of collecting data on their financial situations. In our project we attempt to address this problem by using publicly available satellite images to predict the wealth of a city (or, more generally, a geographic region) based on fundamental features identified in these images and running them through a support vector machine(SVM). Not only would this method be applicable to regions that lack economic data, but could also be applied to cities with a wealth of economic information on a macro level but a dearth on a micro level. For example, cities in America, despite having lots of economic data on state and county levels, could benefit from understanding more granular information in order to improve policy decisions for infrastructure and public support. 


## Related Work

Night-Time Light Data: A Good Proxy Measure for Economic Activity?
Charlotta Mellander, International Business School, Jönköping University, Sweden
José Lobo, School of Human Evolution and Social Change, Arizona State University, USA
Kevin Stolarick, Inclusive Design Research Centre, OCAD University, Canada
Zara Matheson, Former Martin Prosperity Institute, Rotman School of Management, University of Toronto, Canada
http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0139779

Authors of this paper investigated how economic metrics like density, urbanisation, economic growth are related to Night Time Lighting obtained from Satellite Images. They used a "fine grained geo-coded residential and industrial full sample micro-data set for Sweden" and then used correlation analysis and geographically weighted regressions to match it with radiance and saturated light emissions. Interestingly they found that "correlation between NTL and economic activity is strong enough to make it a relatively good proxy for population and establishment density, but the correlation is weaker in relation to wages." Further, they said that they find a "stronger relation between light and density values, than with light and total values. We also find a closer connection between radiance light and economic activity, than with saturated light." 


Finding Poverty in Satellite Images 
Neal Jean, Stanford University, California, USA
Rachel Luo, Stanford University, California, USA
http://cs229.stanford.edu/proj2015/162_report.pdf

This paper uses only day time satellite images because the authors discovered that NTL is almost completely absent in very poor and rural areas, which makes it impossible to gauge economic activity of these regions with night time images. In this study, the authors took the output of a convolutional neural network (CNN), a 4,096-element feature vector, and use these image features along with known survey data from certain parts of Uganda and Tanzania to perform linear regression on continuous wealth measures. Next, they use this model to predict both consumption-based and asset-based wealth measures, and they find that their predictions do in fact "approach survey accuracy."


We take inspiration from both these works and try to extract features and use machine learning techniques that are most feasible for the nature and scale for our project. 


## Features

In order for this approach to work, we need to be able to extract relevant features from the images in order to train our machine learning model. Our model will not be able to predict the wealth of individual houses (i.e., families), but will work on clusters of houses (i.e., neighborhoods and cities) because of the complexity of wealth measurements and tendency for neighborhood to be at a nearly homogeneous economic level. As a result, we will need to extract "cluster" features to process with our SVM.

Features of satellite images that we extract to elucidate the wealth of a region are:
1. Night Time Lighting
2. Percentage of Special Area types
3. Number of Cars

### Night Time Lighting 

This feature has been widely researched and turns out to be a relatively good indicator of economic activity of a region. Many studies have been done to assets to what extent does Night Time Lighting, and its sub-features like radiance and saturation, relate to economic activity and population density. Mellander et al. in their paper "Night-Time Light Data: A Good Proxy Measure for Economic Activity?" say that "We find that the correlation between NTL and economic activity is strong enough to make it a relatively good proxy for population and establishment density, but the correlation is weaker in relation to wages." In this project we wish to use NTL in tandem with our other features to see whether we can predict average wages of a area more accurately. 

We use high resolution Night Images taken by Earth Observatory NASA in 2012. The collection, called Earth at Night 2012, can be found here: http://earthobservatory.nasa.gov/Features/NightLights/page3.php . These are a collection of 9 images: one map-like image of the whole earth of resolution 54000x27000 and 8 regional tiles of resolution 13500x13500. 

To calculate the approximate the light of a location, given its latitude and longitude: 
1. We find it pixel location on the map-like image, 
2. Calculate which tile image that location corresponds to, 
3. Transform the location to the the correct pixel location in the tile image which has higher resolution, 
4. Average the light over the neighboring pixels.

### Percentage of Special Area types(Roads) 

When looking at high and low incomes during the day, it becomes apparent that high income areas tend to have amenities that are not available in low income areas.  These include well paved roads, parks, museums and other cultural centers.  So, it is natural to try to predict the average income of a city based on how developed the city seems to be, and how many attractions there are for residents to take part in.  The Google Static Maps API enable us to extract multiple area types from an image of a city.  These include natural land, man-made land, roads, water, areas of interest (museums and cultural centers), and parks.  We created features from this by taking the percentage of land in each city that belonged to each category.  
 
### Number of Cars

Another proxy for economic activity is the number of cars on the roads. Yes, some cities that are poorer than others will have more cars probably due to higher population density, but certainly cities that few to no cars will be the poorest. In order to extract the number of cars on the satellite-view Google Maps image we use a two-fold method: 
1. Template Matching 
2. Variance in the color of the roads

Template matching is a technique in digital image processing for finding small parts of an image which match a template image. In our case, the template image is that of a car, and we use Canny edge detection to match it to our satellite image with the help of the openCV package (cv2). We take the edges of both the road pictures and the template picture, and compute the high similarity points (using the TM_COEFF_NORMED values), and rotate the template image around to account for different road directions.  Calling cv2.matchtemplate returns a grayscale image of correlation values according to the above function.  From these images (one for every rotation we do), we pick out values that are above a threshold and use those to be the number of car "pixels" in the image.  We then take the percentage of car pixels to be one of the features in the SVM.

We also used the satellite-view to find the variance in the color of the roads. This metric has multiple uses for us.  The foremost is that it gives us an indication of the quality of roads in a city, as well paved roads tend to be uniform, and poorly maintained roads will have dirt and grass growing.  It also hints at the number of cars on the road because more the number of cars implies the presence of varied colors in the region of the road, so we wanted to see what impact this feature would have on the overall predictions. Since we already know exactly where the roads are, from the road-map, we can easily find the variance in the color of the pixels corresponding the to the region of the roads, which would give us the required feature. 

Both these features in tandem would give us a good handle on the condition of automotive travel in different cities.  While we thought this would be a good predictor for American cities, where car travel is popular, we were also concerned that in European cities that are incredibly wealthy yet don't have many car owners, this may be a false herring for the SVM.


## Training

Finally we use a trained Support Vector Machine to classify new data.  We obtained our training data from http://www.city-data.com/ , which features highly detailed data profiles of all cities in the United States. We used Scikit-learn's SVR to train and predict on our features.  After testing all of the kernels we found that the RBF and sigmoid kernels gave us almost constant predictions for every city, so they were not useful. Of the  poly and linear kernels, the linear one gave us the best fit in the middle range, and the poly kernel was able to predict well for outliers, such as Zurich Switzerland (which has the highest average income in the world).  Ultimately we decided that the linear kernel would be the best for us.

In [1]:
# Import the necessary modules
import json, io, math
import numpy as np
import requests
import scipy.ndimage # used for rotating images
import cv2 # a.k.a., OpenCV
from PIL import Image
from sklearn.svm import SVR
from wealth_predictor import WealthPredictor

In [143]:
# Loads in your API_KEY from the config_secret.json file
with io.open("config_secret.json") as cred:
    API_KEY = json.load(cred)["API_KEY"]
    
# Default values used throughout notebook
BASE_URL = "https://maps.googleapis.com/maps/api/staticmap"
MAX_RGB = 255
DEFAULT_SIZE = 600
DEFAULT_LABEL_VISIBILITY = "off"

# Edge template for what a car looks like
CAR_EDGE_TEMPLATE = cv2.Canny(cv2.cvtColor(np.asarray(Image.open("car_template.png")), cv2.COLOR_BGR2GRAY), 10, 300)

# Defines the amount of latitude or longitude per pixel at different zoom levels
# for the Google Static Maps API.
# Multiply by the size of an image in pixels to see how much latitude or longitude
# the image spans
# City zoom = 12; street zoom = 20
ZOOMS = {
    12: (0.06/230, 0.06/175),
    13: (0.02/153, 0.02/116),
    14: (0.008/122, 0.008/122),
    15: (0.008/245, 0.008/187),
    16: (0.003/183, 0.003/140),
    17: (0.002/245, 0.002/186),
    18: (0.001/245, 0.001/186),
    19: (0.0005/245, 0.0005/186),
    20: (0.0002/196, 0.0002/149)
}

EARTH_IMAGE = np.asarray(Image.open("night_images/dnb_land_ocean_ice_geo.tif"))
EARTH_IMAGE_H, EARTH_IMAGE_W, _ = EARTH_IMAGE.shape

# Returns an Image from the content returned by the Google Static Maps API.
# Necessary to first convert to RGBA to preserve alpha layer, but then we
# can discard the layer correctly by converting to RGB
def load_image(content):
    return Image.open(io.BytesIO(content)).convert("RGBA").convert("RGB")

# Returns the corresponding hex code for an RGB color tuple (R, G, B)
def rgb_to_hex(rgb_tuple):
    return "0x%02x%02x%02x" % rgb_tuple

# Creates the Google Static Maps API payload
# Can specify mode, location, zoom, and colors for stylized map
def create_payload(mode, (lat, lon), zoom, params={}, ret_colors=True):
    size = params.get("size", (DEFAULT_SIZE, DEFAULT_SIZE))
    padding = params.get("padding", 0)
    road_color = params.get("road_color", (0, MAX_RGB, 0))
    road_color_hex = rgb_to_hex(road_color)
    man_made_color = params.get("man_made_color", (0, 0, 0))
    man_made_color_hex = rgb_to_hex(man_made_color)
    poi_color = params.get("poi_color", (MAX_RGB, 0, 0))
    poi_color_hex = rgb_to_hex(poi_color)
    water_color = params.get("water_color", (0, 0, MAX_RGB))
    water_color_hex = rgb_to_hex(water_color)
    natural_color = params.get("natural_color", (MAX_RGB, 0, MAX_RGB))
    natural_color_hex = rgb_to_hex(natural_color)
    label_visibility = params.get("label_visibility", DEFAULT_LABEL_VISIBILITY)

    base_payload = [("size", "{}x{}".format(size[0],size[1])), ("key", API_KEY)]
    # Stylize map to make extraction of features easier
    style_payload = [("style", "feature:road|element:geometry|color:{}|visibility:on".format(road_color_hex)),
                     ("style", "feature:landscape.man_made|element:geometry.fill|color:{}".format(man_made_color_hex)),
                     ("style", "element:labels|visibility:{}".format(label_visibility)),
                     ("style", "feature:poi|element:geometry|color:{}".format(poi_color_hex)),
                     ("style", "feature:water|element:geometry|color:{}".format(water_color_hex)),
                     ("style", "feature:landscape.natural|element:geometry.fill|color:{}".format(natural_color_hex))]
    # Cannot style the satellite map
    satellite_payload = base_payload + [("maptype", "satellite")]
    road_payload = base_payload + style_payload + [("maptype", "roadmap")]
    
    if mode == "satellite": payload = satellite_payload 
    elif mode == "road": payload = road_payload
    else: raise ValueError("Unrecognized mode '{}'. Mode can either be 'satellite' or 'road'.".format(mode))
        
    payload += [("zoom", zoom)] + [("center", "{},{}".format(lat, lon))]
    # Important to know what colors are being used for each map feature so later on, 
    # those features can be extracted easier
    colors = {
        "road": np.array(road_color),
        "man_made": np.array(man_made_color), 
        "poi": np.array(poi_color),
        "water": np.array(water_color),
        "natural": np.array(natural_color)
    }
    
    return (payload, colors) if ret_colors else payload
    

# Bottom left, top right corners of city bounding box
# Returns the list of centers of images in an array of latitudes and longitudes
def bounding_box((lat1,lon1), (lat2,lon2), zoom):
    w = lon2 - lon1
    h = lat2 - lat1
    w_per_image = DEFAULT_SIZE * ZOOMS[zoom][1]
    h_per_image = DEFAULT_SIZE * ZOOMS[zoom][0]
    num_width = math.ceil(w / w_per_image)
    num_height = math.ceil(h / h_per_image)
    lons = np.linspace(lon1 + w_per_image/2, lon2 - w_per_image/2, num=num_width)
    lats = np.linspace(lat1 + h_per_image/2, lat2 - h_per_image/2, num=num_height)
    
    return lats, lons

# Requests an image from the Google Static Maps API with the specified payload
def get_image(lat, lon, payload):
    r = requests.get(BASE_URL, params=payload)
    image = load_image(r.content)
    return image

def get_images((lat1,lon1), (lat2,lon2), zoom, modes, ret_colors=True):
    lats, lons = bounding_box((lat1,lon1), (lat2,lon2), zoom)
    num_images = len(lats) * len(lons)
    images = {
        "road": [],
        "satellite": []
    }
    
    # For every center, download the corresponding image in each mode
    for lat in lats:
        for lon in lons:
            for mode in modes:
                payload, colors = create_payload(mode, (lat, lon), zoom)
                images[mode].append( get_image(lat, lon, payload) )
                
    return (images, num_images, colors) if ret_colors else (images, num_images)

# Night Time Satellite Image

# The satellite image uses an equirectangular projection
# so finding the corresponding x, y image coordinate for
# a given latitude and longitude is easy
def find_pixel(lat, lon, width, height):
    x = int((lon + 180) * (width / 360))
    y = int((90 - lat) * (height / 180))
    return (x, y)

# We return a 20-pixel box around the (x,y) position as an approximation
# to the city radius
def find_light(x, y):
    return EARTH_IMAGE[x-10:x+10,y-10:y+10]

# Returns the average luminosity at a latitude and longitude
def average_light(lat, lon):
    pixelx, pixely = find_pixel(lat, lon, EARTH_IMAGE_W, EARTH_IMAGE_H)
    avg_rgb = np.mean(np.mean(find_light(pixelx, pixely), axis=1), axis=0)
    # Luminosity = 0.2126*R + 0.7152*G + 0.0722*B
    return 0.2126 * avg_rgb[0] + 0.7152 * avg_rgb[1] + 0.0722 * avg_rgb[2]

# Returns a boolean array of pixels that are within a tolerance
# level of the given color in the image
def get_pixels_of_color(im_arr, color, tolerance=10):
    lower_bound = color - tolerance
    lower_bound[lower_bound < 0] = 0
    upper_bound = color + tolerance
    upper_bound[upper_bound > MAX_RGB] = MAX_RGB
    return np.all((im_arr >= lower_bound) & (im_arr <= upper_bound), axis=2)

# Given a boolean array indicating where roads are, returns
# the variance in the road pixels from the satellite image
def road_variance(sat_arr, road_only_arr):
    sat_roads = sat_arr[road_only_arr]
    return sum(np.std(sat_roads, axis=1))

# Given an object edge template, returns the number of occurences of the
# object in the image (with some overlap) correcting for angle if given
# an angular granularity larger than 1
def count_object_pixels(img_rgb, obj_edge_template=CAR_EDGE_TEMPLATE, threshold=0.1, angular_granularity=1):
    w, h = obj_edge_template.shape
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray, 100, 200)
    loc = (np.array(0), np.array(0))
    
    for i in range(angular_granularity):
        template = scipy.ndimage.rotate(obj_edge_template, i * 360. / angular_granularity, mode="constant")
        match_coeff = cv2.matchTemplate(img_edges, template, cv2.TM_CCOEFF_NORMED)
        found = np.where(match_coeff > threshold)
        loc = (np.append(loc[0], found[0]), np.append(loc[1], found[1]))
    
    return len(set(zip(*loc)))

# Extracts all features from the road and satellite images
# Features include:
#   - Percentage of pixels for each kind of pixel (road, water, man_made, poi, natural)
#   - Average light
#   - Number of cars
#   - Road variance
def extract_features(road_arr, sat_arr, colors, ((lat1, lon1), (lat2, lon2)), tolerance=10):
    total_pixels = road_arr.shape[0] * road_arr.shape[1]
    road_pixels = {}
    road_pixel_counts = {}
    
    for kind, color in colors.iteritems():
        road_pixels[kind] = get_pixels_of_color(road_arr, color, tolerance=tolerance)
        
    for kind, color_pixels in road_pixels.iteritems():
        road_pixel_counts[kind] = np.sum(color_pixels)
        
    road_var = road_variance(sat_arr, road_pixels["road"])
    avg_light = average_light(lat1 + (lat2 - lat1)/2, lon1 + (lon2 - lon1)/2)
    car_pixels = float(count_object_pixels(sat_arr)) / road_pixel_counts["road"] if road_pixel_counts["road"] != 0 else 0
    color_features = np.array([float(count) / total_pixels for _, count in road_pixel_counts.iteritems()])
    other_features = np.array([road_var / 1000, car_pixels, avg_light])

    return np.concatenate((color_features, other_features))

# For a given bounding box of a city, finds the features for that city with a certain zoom
def get_features_for_city(((lat1, lon1), (lat2, lon2)), zoom=15):
    modes = ["road", "satellite"]
    images, n, colors = get_images((lat1, lon1), (lat2, lon2), zoom, modes)
    # n pictures, 8 features
    features = np.zeros((n, 8))
    
    for i in xrange(n):
        road_arr = np.asarray(images["road"][i])
        sat_arr = np.asarray(images["satellite"][i])
        features[i] = extract_features(road_arr, sat_arr, colors, ((lat1, lon1), (lat2, lon2)))
    
    # Average over all features (except for road variance) from all of the images to get the
    # features for the entire city
    new_features = np.average(features, axis=0)
    new_features[0] = np.sum(features[:,0])
    return new_features

# User-friendly interface for finding the wealth of a city (defined as a bounding box of latitude and longitude)
class WealthPredictor:
    # Zoom will affect how long the predictor takes to run as well as the granularity of the features,
    # especially number of cars and road variance
    def __init__(self, zoom=15):
        if zoom < 12 or zoom > 20: raise ValueError("Zoom must be between 12 and 20 (inclusive)")
        self.zoom = zoom
        self.classifier = SVR(kernel="linear")
        
    def train(self, train_coords, train_labels):
        train_features = np.array([get_features_for_city(coord, zoom=self.zoom) for coord in train_coords])
        self.classifier.fit(train_features, train_labels)
        
        # print "Training features:", train_features
        # print "Coefficients:", self.classifier.coef_
    
    def predict(self, test_coords):
        test_features = np.array([get_features_for_city(coord) for coord in test_coords])
        return self.classifier.predict(test_features)
    
        # print "Test features:", test_features
    
    def score(self, test_coords, test_labels):
        test_features = np.array([get_features_for_city(coord) for coord in test_coords])
        return self.classifier.score(test_features, test_labels)

Training...
Training features: [[ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]]
Coefficients: [[  0.00000000e+00   0.00000000e+00   4.44089210e-16]]
Done!


In [144]:
    # City labels just for reference
training_places = ["Summerville, GA", "San Francisco, CA", "DC", "Blackwater, AZ", 
                   "Sneedville, TN", "Newton, MA", "Mamou, LA",
                   "Kirkland, WA"]
training_coords = [((34.472709, -85.365401), (34.490396, -85.328837)),
                   ((37.710978, -122.500087), (37.802606, -122.404110)),
                   ((38.819597, -77.160145), (38.983113, -76.912953)),
                   ((33.000521, -111.664691), (33.045859, -111.568733)),
                   ((36.513680, -83.234828), (36.555887, -83.166506)),
                   ((42.288678, -71.267065), (42.366094, -71.160635)), 
                   ((30.624943, -92.425908), (30.644181, -92.410887)),
                   ((47.653627, -122.204879), (47.722267, -122.178787))]
training_labels = [17881, 51686, 45477, 7268, 10958, 68122, 14126, 50991]

print "Training WealthPredictor..."
wp = WealthPredictor(13)
wp.train(training_coords, training_labels)
print "Done!"

# Again, city labels just for reference
training_places = ["University Park, NM", "Pittsburgh, PA", "Tenafly, NJ"]
test_coords = [((32.263894, -106.765491), (32.285883, -106.739656)),
               ((40.417268, -80.036749), (40.418268, -80.035749)),
               ((40.901511, -73.982347), (40.936339, -73.930420))]
test_labels = [5520, 28176, 73846]

print "Scoring test coordinates..."
print wp.predict(test_coords)
print "Done!"

Scoring test coordinates...
Test features: [[ 0.      0.      3.5242]
 [ 0.      0.      3.5242]
 [ 0.      0.      3.5242]]
[ 31679.  31679.  31679.]
-0.0215170741987
Done!


## Results

## Improvement